# Architectures

In [38]:
import numpy as np
import tensorflow as tf
from collections import OrderedDict

In [36]:
def conv_layer(x,scope,filters=12,kernel_size=5,norm=True,avtive=True):
    
    y = tf.layers.conv2d(x, filters=filters,
                            kernel_size=kernel_size,
                            padding='SAME',
                            kernel_initializer=tf.contrib.layers.xavier_initializer(),
                            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.05),
                            bias_initializer=tf.constant_initializer(0.1),
                            name=scope+'_conv')
    if norm:    y = tf.layers.batch_normalization(y,name=scope+'_norm')
    if avtive:    y = tf.nn.relu(y,name=scope+'_act')
    
    return y

def architecture(x_in,n_layers,n_channel=1,res=3):
    
    layers = [x_in]
    for i in range(n_layers-1):
        layers.append(conv_layer(layers[-1],scope='layer_'+str(i+1)))
        print(layers[-1])
        
        if res:
            if (((i-1)%res==0) & (i>1)):
                layers.append(layers[-1]+layers[i-res])
                print('Res layer',i,'+',i-res,':')
                print(layers[-1])
        
    layers.append(conv_layer(layers[-1],scope='layer_'+str(n_layers),filters=n_channel,avtive=0))
    print(layers[-1])

    return layers

In [37]:
tf.reset_default_graph()

x_in = tf.placeholder(tf.float32,[None,20,20,3])
layers = architecture(x_in,5,res=2)
x_out = layers[-1]

Tensor("layer_1_act:0", shape=(?, 20, 20, 12), dtype=float32)
Tensor("layer_2_act:0", shape=(?, 20, 20, 12), dtype=float32)
Tensor("layer_3_act:0", shape=(?, 20, 20, 12), dtype=float32)
Tensor("layer_4_act:0", shape=(?, 20, 20, 12), dtype=float32)
Res layer 3 + 1 :
Tensor("add:0", shape=(?, 20, 20, 12), dtype=float32)
Tensor("layer_5_norm/FusedBatchNorm:0", shape=(?, 20, 20, 1), dtype=float32)


# Semi U-net

In [44]:
def conv2d(x,num_outputs,kernel_size,stride,padding='VALID',activation_fn=tf.nn.relu,trainable=True,scope=None):
    return tf.contrib.layers.conv2d(
        x,
        num_outputs=num_outputs,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        data_format='NHWC',
        rate=1,
        activation_fn=activation_fn,
        normalizer_fn=None,
        normalizer_params=None,
        weights_regularizer=None,
        biases_initializer=tf.constant_initializer(0.1),
        biases_regularizer=None,
        reuse=None,
        variables_collections=None,
        outputs_collections=None,
        trainable=trainable,
        scope=scope
    )

def deconv2d(x,num_outputs,kernel_size,stride,padding='VALID',activation_fn=tf.nn.relu,trainable=True,scope=None):
    return tf.contrib.layers.conv2d_transpose(
        x,
        num_outputs=num_outputs,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        data_format='NHWC',
        activation_fn=activation_fn,
        normalizer_fn=None,
        normalizer_params=None,
        weights_regularizer=None,
        biases_initializer=tf.constant_initializer(0.1),
        biases_regularizer=None,
        reuse=None,
        variables_collections=None,
        outputs_collections=None,
        trainable=trainable,
        scope=scope
    )

def architecture(x_in,drop_out_rate=0.3):

#    x = tf.placeholder(dtype=tf.float32, shape=(20,l,l,5))
    x = x_in
    l = x.get_shape().as_list()[1]
    np2 = int(np.log(l)/np.log(2))
    main = 2**np2
    n_residue = l-main
    res = []
    nt = 1
    while n_residue>np.sum(res)-len(res):
        res.append(nt)
        nt += 8
    res = res[1:-1]
    res.append(n_residue-np.sum(res)+len(res)+1)
        
    print('===================================================')
    print('===================== ENCODER =====================')
    print('===================================================')
    print(x)
    layers = OrderedDict()
    nr = len(res)
    i_layer = 0
    n_layers = 0
    for i in range(nr):
        i_layer += 1
        ks = res[nr-i-1]
        x = conv2d(x,6,[ks,ks],[1,1],padding='VALID',scope='encode/deconv_'+str(i_layer))
        x = tf.layers.batch_normalization(x,name='encode/batch_'+str(i_layer))
        x = tf.layers.dropout(x,rate=drop_out_rate,training=True,name='encode/dropout_'+str(i_layer))
        x = tf.nn.relu(x,name='encode/relu_'+str(i_layer))
        layers['encode_'+str(i_layer)] = x
        print(x)
        
    for i in range(np2-5):
        i_layer += 1
        x = conv2d(x,6,[5,5],[2,2],padding='SAME',scope='encode/deconv_'+str(i_layer))
        x = tf.layers.batch_normalization(x,name='encode/batch_'+str(i_layer))
        x = tf.layers.dropout(x,rate=drop_out_rate,training=True,name='encode/dropout_'+str(i_layer))
        x = tf.nn.relu(x,name='encode/relu_'+str(i_layer))
        layers['encode_'+str(i_layer)] = x
        print(x)
    n_layers = i_layer
        
    print('')
    print('===================================================')
    print('===================== DECODER =====================')
    print('===================================================')

    i_layer = 0
    for i in range(np2-5):
        i_layer += 1
        x = deconv2d(x,6,[5,5],[2,2],padding='SAME',scope='decode/deconv_'+str(i_layer))
        x = tf.layers.batch_normalization(x,name='decode/batch_'+str(i_layer))
        x = tf.layers.dropout(x,rate=drop_out_rate,training=True,name='decode/dropout_'+str(i_layer))
        x = tf.nn.relu(x,name='decode/relu_'+str(i_layer))
        layers['decode_'+str(i_layer)] = x
        print(x)

    for i in range(nr):
        i_layer += 1
        ks = res[i]
        x = deconv2d(x,6,[ks,ks],[1,1],padding='VALID',scope='decode/deconv_'+str(i_layer))
        x = tf.layers.batch_normalization(x,name='decode/batch_'+str(i_layer))
        x = tf.layers.dropout(x,rate=drop_out_rate,training=True,name='decode/dropout_'+str(i_layer))
        x = tf.nn.relu(x,name='decode/relu_'+str(i_layer))
        layers['decode_'+str(i_layer)] = x
        print(x)
    
    return [layers,x]

In [45]:
tf.reset_default_graph()

x_in = tf.placeholder(tf.float32,[None,400,400,3])
layers = architecture(x_in)
x_out = layers[-1]

===================== ENCODER =====================
Tensor("Placeholder:0", shape=(?, 400, 400, 3), dtype=float32)
Tensor("encode/relu_1:0", shape=(?, 376, 376, 6), dtype=float32)
Tensor("encode/relu_2:0", shape=(?, 336, 336, 6), dtype=float32)
Tensor("encode/relu_3:0", shape=(?, 304, 304, 6), dtype=float32)
Tensor("encode/relu_4:0", shape=(?, 280, 280, 6), dtype=float32)
Tensor("encode/relu_5:0", shape=(?, 264, 264, 6), dtype=float32)
Tensor("encode/relu_6:0", shape=(?, 256, 256, 6), dtype=float32)
Tensor("encode/relu_7:0", shape=(?, 128, 128, 6), dtype=float32)
Tensor("encode/relu_8:0", shape=(?, 64, 64, 6), dtype=float32)
Tensor("encode/relu_9:0", shape=(?, 32, 32, 6), dtype=float32)

===================== DECODER =====================
Tensor("decode/relu_1:0", shape=(?, 64, 64, 6), dtype=float32)
Tensor("decode/relu_2:0", shape=(?, 128, 128, 6), dtype=float32)
Tensor("decode/relu_3:0", shape=(?, 256, 256, 6), dtype=float32)
Tensor("decode/relu_4:0", shape=(?, 264, 264, 6), dtype=fl